# Tensorflow birdcall vggish

## About

In this notebook, I try to implement vggish model of tensorflow. 


I want to thanks following kernel authors for such a nice public kernel
https://www.kaggle.com/ttahara/training-birdsong-baseline-resnest50-fast
https://www.kaggle.com/hidehisaarai1213/inference-pytorch-birdcall-resnet-baseline


## Prepare

### import libraries

In [ ]:
!pip install numpy resampy tensorflow tf_slim six soundfile

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
# import efficientnet.tfkeras as efn
from kaggle_datasets import KaggleDatasets

import os
import shutil
import numpy as np

import tensorflow as tf
from tensorflow import keras

from pathlib import Path
from IPython.display import display, Audio

seed_value = 42
tf.random.set_seed(seed_value)

In [ ]:
DEVICE = "tpu"
if DEVICE == "tpu":
    print("Connecting for tpu")
    try :
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print("Running on tpu", tpu.master())
    except ValueError:
        print("could not connect TPU")
        tpu = None
    if tpu:
        try:
            print("Initialize TPU...")
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print('TPU initialized')
        except _:
            print('Failed to initialize TPU!')
    else:
        DEVICE = 'GPU'
if DEVICE != 'tpu':
    print('Using default strategy for CPU and single GPU')
    strategy = tf.distribute.get_strategy()

if DEVICE == 'GPU':
    print('Num GPUs Available: ',
          len(tf.config.experimental.list_physical_devices('GPU')))

print('REPLICAS: ', strategy.num_replicas_in_sync)
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
import os
import gc
import time
import shutil
import random
import warnings
import typing as tp
from pathlib import Path
from contextlib import contextmanager

import yaml
from joblib import delayed, Parallel

import cv2
import librosa
import audioread
import soundfile as sf

import numpy as np
import pandas as pd

from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

# import pytorch_pfn_extras as ppe
# from pytorch_pfn_extras.training import extensions as ppe_extensions

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

In [ ]:
NUM_FRAMES = 96  # Frames in input mel-spectrogram patch.
NUM_BANDS = 64  # Frequency bands in input mel-spectrogram patch.
EMBEDDING_SIZE = 128  # Size of embedding layer.

# Hyperparameters used in feature and example generation.
SAMPLE_RATE = 16000
STFT_WINDOW_LENGTH_SECONDS = 0.025
STFT_HOP_LENGTH_SECONDS = 0.010
NUM_MEL_BINS = NUM_BANDS
MEL_MIN_HZ = 125
MEL_MAX_HZ = 7500
LOG_OFFSET = 0.01  # Offset used for stabilized log of input mel-spectrogram.
EXAMPLE_WINDOW_SECONDS = 0.96  # Each example contains 96 10ms frames
EXAMPLE_HOP_SECONDS = 0.96     # with zero overlap.

# Parameters used for embedding postprocessing.
PCA_EIGEN_VECTORS_NAME = 'pca_eigen_vectors'
PCA_MEANS_NAME = 'pca_means'
QUANTIZE_MIN_VAL = -2.0
QUANTIZE_MAX_VAL = +2.0

# Hyperparameters used in training.
INIT_STDDEV = 0.01  # Standard deviation used to initialize weights.
LEARNING_RATE = 1e-4  # Learning rate for the Adam optimizer.
ADAM_EPSILON = 1e-8  # Epsilon for the Adam optimizer.

# Names of ops, tensors, and features.
INPUT_OP_NAME = 'vggish/input_features'
INPUT_TENSOR_NAME = INPUT_OP_NAME + ':0'
OUTPUT_OP_NAME = 'vggish/embedding'
OUTPUT_TENSOR_NAME = OUTPUT_OP_NAME + ':0'
AUDIO_EMBEDDING_FEATURE_NAME = 'audio_embedding'

In [ ]:
import numpy as np


def frame(data, window_length, hop_length):
  """Convert array into a sequence of successive possibly overlapping frames.
  An n-dimensional array of shape (num_samples, ...) is converted into an
  (n+1)-D array of shape (num_frames, window_length, ...), where each frame
  starts hop_length points after the preceding one.
  This is accomplished using stride_tricks, so the original data is not
  copied.  However, there is no zero-padding, so any incomplete frames at the
  end are not included.
  Args:
    data: np.array of dimension N >= 1.
    window_length: Number of samples in each frame.
    hop_length: Advance (in samples) between each window.
  Returns:
    (N+1)-D np.array with as many rows as there are complete frames that can be
    extracted.
  """
  num_samples = data.shape[0]
  num_frames = 1 + int(np.floor((num_samples - window_length) / hop_length))
  shape = (num_frames, window_length) + data.shape[1:]
  strides = (data.strides[0] * hop_length,) + data.strides
  return np.lib.stride_tricks.as_strided(data, shape=shape, strides=strides)


def periodic_hann(window_length):
  """Calculate a "periodic" Hann window.
  The classic Hann window is defined as a raised cosine that starts and
  ends on zero, and where every value appears twice, except the middle
  point for an odd-length window.  Matlab calls this a "symmetric" window
  and np.hanning() returns it.  However, for Fourier analysis, this
  actually represents just over one cycle of a period N-1 cosine, and
  thus is not compactly expressed on a length-N Fourier basis.  Instead,
  it's better to use a raised cosine that ends just before the final
  zero value - i.e. a complete cycle of a period-N cosine.  Matlab
  calls this a "periodic" window. This routine calculates it.
  Args:
    window_length: The number of points in the returned window.
  Returns:
    A 1D np.array containing the periodic hann window.
  """
  return 0.5 - (0.5 * np.cos(2 * np.pi / window_length *
                             np.arange(window_length)))


def stft_magnitude(signal, fft_length,
                   hop_length=None,
                   window_length=None):
  """Calculate the short-time Fourier transform magnitude.
  Args:
    signal: 1D np.array of the input time-domain signal.
    fft_length: Size of the FFT to apply.
    hop_length: Advance (in samples) between each frame passed to FFT.
    window_length: Length of each block of samples to pass to FFT.
  Returns:
    2D np.array where each row contains the magnitudes of the fft_length/2+1
    unique values of the FFT for the corresponding frame of input samples.
  """
  frames = frame(signal, window_length, hop_length)
  # Apply frame window to each frame. We use a periodic Hann (cosine of period
  # window_length) instead of the symmetric Hann of np.hanning (period
  # window_length-1).
  window = periodic_hann(window_length)
  windowed_frames = frames * window
  return np.abs(np.fft.rfft(windowed_frames, int(fft_length)))


# Mel spectrum constants and functions.
_MEL_BREAK_FREQUENCY_HERTZ = 700.0
_MEL_HIGH_FREQUENCY_Q = 1127.0


def hertz_to_mel(frequencies_hertz):
  """Convert frequencies to mel scale using HTK formula.
  Args:
    frequencies_hertz: Scalar or np.array of frequencies in hertz.
  Returns:
    Object of same size as frequencies_hertz containing corresponding values
    on the mel scale.
  """
  return _MEL_HIGH_FREQUENCY_Q * np.log(
      1.0 + (frequencies_hertz / _MEL_BREAK_FREQUENCY_HERTZ))


def spectrogram_to_mel_matrix(num_mel_bins=20,
                              num_spectrogram_bins=129,
                              audio_sample_rate=8000,
                              lower_edge_hertz=125.0,
                              upper_edge_hertz=3800.0):
  """Return a matrix that can post-multiply spectrogram rows to make mel.
  Returns a np.array matrix A that can be used to post-multiply a matrix S of
  spectrogram values (STFT magnitudes) arranged as frames x bins to generate a
  "mel spectrogram" M of frames x num_mel_bins.  M = S A.
  The classic HTK algorithm exploits the complementarity of adjacent mel bands
  to multiply each FFT bin by only one mel weight, then add it, with positive
  and negative signs, to the two adjacent mel bands to which that bin
  contributes.  Here, by expressing this operation as a matrix multiply, we go
  from num_fft multiplies per frame (plus around 2*num_fft adds) to around
  num_fft^2 multiplies and adds.  However, because these are all presumably
  accomplished in a single call to np.dot(), it's not clear which approach is
  faster in Python.  The matrix multiplication has the attraction of being more
  general and flexible, and much easier to read.
  Args:
    num_mel_bins: How many bands in the resulting mel spectrum.  This is
      the number of columns in the output matrix.
    num_spectrogram_bins: How many bins there are in the source spectrogram
      data, which is understood to be fft_size/2 + 1, i.e. the spectrogram
      only contains the nonredundant FFT bins.
    audio_sample_rate: Samples per second of the audio at the input to the
      spectrogram. We need this to figure out the actual frequencies for
      each spectrogram bin, which dictates how they are mapped into mel.
    lower_edge_hertz: Lower bound on the frequencies to be included in the mel
      spectrum.  This corresponds to the lower edge of the lowest triangular
      band.
    upper_edge_hertz: The desired top edge of the highest frequency band.
  Returns:
    An np.array with shape (num_spectrogram_bins, num_mel_bins).
  Raises:
    ValueError: if frequency edges are incorrectly ordered or out of range.
  """
  nyquist_hertz = audio_sample_rate / 2.
  if lower_edge_hertz < 0.0:
    raise ValueError("lower_edge_hertz %.1f must be >= 0" % lower_edge_hertz)
  if lower_edge_hertz >= upper_edge_hertz:
    raise ValueError("lower_edge_hertz %.1f >= upper_edge_hertz %.1f" %
                     (lower_edge_hertz, upper_edge_hertz))
  if upper_edge_hertz > nyquist_hertz:
    raise ValueError("upper_edge_hertz %.1f is greater than Nyquist %.1f" %
                     (upper_edge_hertz, nyquist_hertz))
  spectrogram_bins_hertz = np.linspace(0.0, nyquist_hertz, num_spectrogram_bins)
  spectrogram_bins_mel = hertz_to_mel(spectrogram_bins_hertz)
  # The i'th mel band (starting from i=1) has center frequency
  # band_edges_mel[i], lower edge band_edges_mel[i-1], and higher edge
  # band_edges_mel[i+1].  Thus, we need num_mel_bins + 2 values in
  # the band_edges_mel arrays.
  band_edges_mel = np.linspace(hertz_to_mel(lower_edge_hertz),
                               hertz_to_mel(upper_edge_hertz), num_mel_bins + 2)
  # Matrix to post-multiply feature arrays whose rows are num_spectrogram_bins
  # of spectrogram values.
  mel_weights_matrix = np.empty((num_spectrogram_bins, num_mel_bins))
  for i in range(num_mel_bins):
    lower_edge_mel, center_mel, upper_edge_mel = band_edges_mel[i:i + 3]
    # Calculate lower and upper slopes for every spectrogram bin.
    # Line segments are linear in the *mel* domain, not hertz.
    lower_slope = ((spectrogram_bins_mel - lower_edge_mel) /
                   (center_mel - lower_edge_mel))
    upper_slope = ((upper_edge_mel - spectrogram_bins_mel) /
                   (upper_edge_mel - center_mel))
    # .. then intersect them with each other and zero.
    mel_weights_matrix[:, i] = np.maximum(0.0, np.minimum(lower_slope,
                                                          upper_slope))
  # HTK excludes the spectrogram DC bin; make sure it always gets a zero
  # coefficient.
  mel_weights_matrix[0, :] = 0.0
  return mel_weights_matrix


def log_mel_spectrogram(data,
                        audio_sample_rate=8000,
                        log_offset=0.0,
                        window_length_secs=0.025,
                        hop_length_secs=0.010,
                        **kwargs):
  """Convert waveform to a log magnitude mel-frequency spectrogram.
  Args:
    data: 1D np.array of waveform data.
    audio_sample_rate: The sampling rate of data.
    log_offset: Add this to values when taking log to avoid -Infs.
    window_length_secs: Duration of each window to analyze.
    hop_length_secs: Advance between successive analysis windows.
    **kwargs: Additional arguments to pass to spectrogram_to_mel_matrix.
  Returns:
    2D np.array of (num_frames, num_mel_bins) consisting of log mel filterbank
    magnitudes for successive frames.
  """
  window_length_samples = int(round(audio_sample_rate * window_length_secs))
  hop_length_samples = int(round(audio_sample_rate * hop_length_secs))
  fft_length = 2 ** int(np.ceil(np.log(window_length_samples) / np.log(2.0)))
  spectrogram = stft_magnitude(
      data,
      fft_length=fft_length,
      hop_length=hop_length_samples,
      window_length=window_length_samples)
  mel_spectrogram = np.dot(spectrogram, spectrogram_to_mel_matrix(
      num_spectrogram_bins=spectrogram.shape[1],
      audio_sample_rate=audio_sample_rate, **kwargs))
  return np.log(mel_spectrogram + log_offset)

In [ ]:
def _remove_silent_frames(audio, top_db):
    indices = librosa.effects.split(audio, top_db=top_db)

    trimmed_audio = []
    for index in indices:
        trimmed_audio.append(torch.tensor(audio[index[0]: index[1]]))

    return np.concatenate(trimmed_audio, axis=0)

In [ ]:
import numpy as np
import resampy
import librosa

PERIOD = 10

try:
  import soundfile as sf

  def wav_read(wav_file):
    wav_data, sr = sf.read(wav_file)
    return wav_data, sr

except ImportError:

  def wav_read(wav_file):
    raise NotImplementedError('WAV file reading requires soundfile package.')

def _remove_silent_frames(audio, top_db):
    indices = librosa.effects.split(audio, top_db=top_db)

    trimmed_audio = []
    for index in indices:
        trimmed_audio.append(torch.tensor(audio[index[0]: index[1]]))

    return np.concatenate(trimmed_audio, axis=0)

def waveform_to_examples(data, sample_rate):
  """Converts audio waveform into an array of examples for VGGish.
  Args:
    data: np.array of either one dimension (mono) or two dimensions
      (multi-channel, with the outer dimension representing channels).
      Each sample is generally expected to lie in the range [-1.0, +1.0],
      although this is not required.
    sample_rate: Sample rate of data.
  Returns:
    3-D np.array of shape [num_examples, num_frames, num_bands] which represents
    a sequence of examples, each of which contains a patch of log mel
    spectrogram, covering num_frames frames of audio and num_bands mel frequency
    bands, where the frame length is vggish_params.STFT_HOP_LENGTH_SECONDS.
  """
  # Convert to mono.
  if len(data.shape) > 1:
    data = np.mean(data, axis=1)
  # Resample to the rate assumed by VGGish.
  if sample_rate != SAMPLE_RATE:
    data = resampy.resample(data, sample_rate, SAMPLE_RATE)

  # Compute log mel spectrogram features.
  log_mel = log_mel_spectrogram(
      data,
      audio_sample_rate= SAMPLE_RATE,
      log_offset= LOG_OFFSET,
      window_length_secs= STFT_WINDOW_LENGTH_SECONDS,
      hop_length_secs= STFT_HOP_LENGTH_SECONDS,
      num_mel_bins= NUM_MEL_BINS,
      lower_edge_hertz= MEL_MIN_HZ,
      upper_edge_hertz= MEL_MAX_HZ)

  # Frame features into examples.
  features_sample_rate = 1.0 /  STFT_HOP_LENGTH_SECONDS
  example_window_length = int(round(
      EXAMPLE_WINDOW_SECONDS * features_sample_rate))
  example_hop_length = int(round(
      EXAMPLE_HOP_SECONDS * features_sample_rate))
  log_mel_examples = frame(
      log_mel,
      window_length=example_window_length,
      hop_length=example_hop_length)
  return log_mel_examples


def wavfile_to_examples(wav_file):
  """Convenience wrapper around waveform_to_examples() for a common WAV format.
  Args:
    wav_file: String path to a file, or a file-like object. The file
    is assumed to contain WAV audio data with signed 16-bit PCM samples.
  Returns:
    See waveform_to_examples.
  """
  wav_data, sr = wav_read(wav_file)
  wav_data = _remove_silent_frames(wav_data, top_db=40)
  effective_length = sr * PERIOD  
  len_y = len(wav_data)

  if len_y < effective_length:
                new_y = np.zeros(effective_length, dtype=wav_data.dtype)
                start = np.random.randint(effective_length - len_y)
                new_y[start:start + len_y] = wav_data
                wav_data = new_y.astype(np.float32)
  elif len_y > effective_length:
                start = np.random.randint(len_y - effective_length)
                wav_data = wav_data[start:start + effective_length].astype(np.float32)
  else:
                wav_data = wav_data.astype(np.float32)
                
  wav_data = wav_data * 32768
  wav_data = wav_data.astype(np.int16)
    
  assert wav_data.dtype == np.int16, 'Bad sample type: %r' % wav_data.dtype
  samples = wav_data / 32768.0  # Convert to [-1.0, +1.0]
  return waveform_to_examples(samples, sr)

In [ ]:
import tensorflow.compat.v1 as tf
import tf_slim as slim




def define_vggish_slim(features_tensor=None, training=False):
  """Defines the VGGish TensorFlow model.
  All ops are created in the current default graph, under the scope 'vggish/'.
  The input is either a tensor passed in via the optional 'features_tensor'
  argument or a placeholder created below named 'vggish/input_features'. The
  input is expected to have dtype float32 and shape [batch_size, num_frames,
  num_bands] where batch_size is variable and num_frames and num_bands are
  constants, and [num_frames, num_bands] represents a log-mel-scale spectrogram
  patch covering num_bands frequency bands and num_frames time frames (where
  each frame step is usually 10ms). This is produced by computing the stabilized
  log(mel-spectrogram + params.LOG_OFFSET).  The output is a tensor named
  'vggish/embedding' which produces the pre-activation values of a 128-D
  embedding layer, which is usually the penultimate layer when used as part of a
  full model with a final classifier layer.
  Args:
    features_tensor: If not None, the tensor containing the input features.
      If None, a placeholder input is created.
    training: If true, all parameters are marked trainable.
  Returns:
    The op 'vggish/embeddings'.
  """
  # Defaults:
  # - All weights are initialized to N(0, INIT_STDDEV).
  # - All biases are initialized to 0.
  # - All activations are ReLU.
  # - All convolutions are 3x3 with stride 1 and SAME padding.
  # - All max-pools are 2x2 with stride 2 and SAME padding.
  with slim.arg_scope([slim.conv2d, slim.fully_connected],
                      weights_initializer=tf.truncated_normal_initializer(
                          stddev=INIT_STDDEV),
                      biases_initializer=tf.zeros_initializer(),
                      activation_fn=tf.nn.relu,
                      trainable=training), \
       slim.arg_scope([slim.conv2d],
                      kernel_size=[3, 3], stride=1, padding='SAME'), \
       slim.arg_scope([slim.max_pool2d],
                      kernel_size=[2, 2], stride=2, padding='SAME'), \
       tf.variable_scope('vggish'):
    # Input: a batch of 2-D log-mel-spectrogram patches.
    if features_tensor is None:
      features_tensor = tf.placeholder(
          tf.float32, shape=(None, NUM_FRAMES, NUM_BANDS),
          name='input_features')
    # Reshape to 4-D so that we can convolve a batch with conv2d().
    net = tf.reshape(features_tensor,
                     [-1, NUM_FRAMES, NUM_BANDS, 1])

    # The VGG stack of alternating convolutions and max-pools.
    net = slim.conv2d(net, 64, scope='conv1')
    net = slim.max_pool2d(net, scope='pool1')
    net = slim.conv2d(net, 128, scope='conv2')
    net = slim.max_pool2d(net, scope='pool2')
    net = slim.repeat(net, 2, slim.conv2d, 256, scope='conv3')
    net = slim.max_pool2d(net, scope='pool3')
    net = slim.repeat(net, 2, slim.conv2d, 512, scope='conv4')
    net = slim.max_pool2d(net, scope='pool4')

    # Flatten before entering fully-connected layers
    net = slim.flatten(net)
    net = slim.repeat(net, 2, slim.fully_connected, 4096, scope='fc1')
    # The embedding layer.
    net = slim.fully_connected(net, EMBEDDING_SIZE, scope='fc2',
                               activation_fn=None)
    return tf.identity(net, name='embedding')


def load_vggish_slim_checkpoint(session, checkpoint_path):
  """Loads a pre-trained VGGish-compatible checkpoint.
  This function can be used as an initialization function (referred to as
  init_fn in TensorFlow documentation) which is called in a Session after
  initializating all variables. When used as an init_fn, this will load
  a pre-trained checkpoint that is compatible with the VGGish model
  definition. Only variables defined by VGGish will be loaded.
  Args:
    session: an active TensorFlow session.
    checkpoint_path: path to a file containing a checkpoint that is
      compatible with the VGGish model definition.
  """
  # Get the list of names of all VGGish variables that exist in
  # the checkpoint (i.e., all inference-mode VGGish variables).
  with tf.Graph().as_default():
    define_vggish_slim(training=False)
    vggish_var_names = [v.name for v in tf.global_variables()]

  # Get the list of all currently existing variables that match
  # the list of variable names we just computed.
  vggish_vars = [v for v in tf.global_variables() if v.name in vggish_var_names]

  # Use a Saver to restore just the variables selected above.
  saver = tf.train.Saver(vggish_vars, name='vggish_load_pretrained',
                         write_version=1)
  saver.restore(session, checkpoint_path)

### read data

In [ ]:
ROOT = Path.cwd().parent
INPUT_ROOT = ROOT / "input"
RAW_DATA = INPUT_ROOT / "birdsong-recognition"
TRAIN_AUDIO_DIR = RAW_DATA / "train_audio"


TRAIN_RESAMPLED_AUDIO_DIRS = [
  INPUT_ROOT / "birdsong-resampled-train-audio-{:0>2}".format(i)  for i in range(5)
]
train = pd.read_csv(TRAIN_RESAMPLED_AUDIO_DIRS[0] / "train_mod.csv")
TEST_AUDIO_DIR = RAW_DATA / "test_audio"

### settings

In [ ]:
settings_str = """
globals:
  seed: 1213
  device: cuda
  num_epochs: 5
  output_dir: /kaggle/training_output/
  use_fold: 0
  target_sr: 32000

dataset:
  name: SpectrogramDataset
  params:
    img_size: 224
    melspectrogram_parameters:
      n_mels: 128
      fmin: 20
      fmax: 16000
    
split:
  name: StratifiedKFold
  params:
    n_splits: 5
    random_state: 42
    shuffle: True

loader:
  train:
    batch_size: 32
    shuffle: True
    num_workers: 2
    pin_memory: True
    drop_last: True
  val:
    batch_size: 32
    shuffle: False
    num_workers: 2
    pin_memory: True
    drop_last: False

model:
  name: resnest50_fast_1s1x64d
  params:
    pretrained: True
    n_classes: 264

loss:
  name: BCEWithLogitsLoss
  params: {}

optimizer:
  name: Adam
  params:
    lr: 0.001

scheduler:
  name: CosineAnnealingLR
  params:
    T_max: 10
"""

In [ ]:
settings = yaml.safe_load(settings_str)

In [ ]:
if not torch.cuda.is_available():
    settings["globals"]["device"] = "cpu"

In [ ]:
for k, v in settings.items():
    print("[{}]".format(k))
    print(v)

## Definition

### Dataset
* forked from: https://github.com/koukyo1994/kaggle-birdcall-resnet-baseline-training/blob/master/src/dataset.py
* modified partialy


In [ ]:
BIRD_CODE = {
    'aldfly': 0, 'ameavo': 1, 'amebit': 2, 'amecro': 3, 'amegfi': 4,
    'amekes': 5, 'amepip': 6, 'amered': 7, 'amerob': 8, 'amewig': 9,
    'amewoo': 10, 'amtspa': 11, 'annhum': 12, 'astfly': 13, 'baisan': 14,
    'baleag': 15, 'balori': 16, 'banswa': 17, 'barswa': 18, 'bawwar': 19,
    'belkin1': 20, 'belspa2': 21, 'bewwre': 22, 'bkbcuc': 23, 'bkbmag1': 24,
    'bkbwar': 25, 'bkcchi': 26, 'bkchum': 27, 'bkhgro': 28, 'bkpwar': 29,
    'bktspa': 30, 'blkpho': 31, 'blugrb1': 32, 'blujay': 33, 'bnhcow': 34,
    'boboli': 35, 'bongul': 36, 'brdowl': 37, 'brebla': 38, 'brespa': 39,
    'brncre': 40, 'brnthr': 41, 'brthum': 42, 'brwhaw': 43, 'btbwar': 44,
    'btnwar': 45, 'btywar': 46, 'buffle': 47, 'buggna': 48, 'buhvir': 49,
    'bulori': 50, 'bushti': 51, 'buwtea': 52, 'buwwar': 53, 'cacwre': 54,
    'calgul': 55, 'calqua': 56, 'camwar': 57, 'cangoo': 58, 'canwar': 59,
    'canwre': 60, 'carwre': 61, 'casfin': 62, 'caster1': 63, 'casvir': 64,
    'cedwax': 65, 'chispa': 66, 'chiswi': 67, 'chswar': 68, 'chukar': 69,
    'clanut': 70, 'cliswa': 71, 'comgol': 72, 'comgra': 73, 'comloo': 74,
    'commer': 75, 'comnig': 76, 'comrav': 77, 'comred': 78, 'comter': 79,
    'comyel': 80, 'coohaw': 81, 'coshum': 82, 'cowscj1': 83, 'daejun': 84,
    'doccor': 85, 'dowwoo': 86, 'dusfly': 87, 'eargre': 88, 'easblu': 89,
    'easkin': 90, 'easmea': 91, 'easpho': 92, 'eastow': 93, 'eawpew': 94,
    'eucdov': 95, 'eursta': 96, 'evegro': 97, 'fiespa': 98, 'fiscro': 99,
    'foxspa': 100, 'gadwal': 101, 'gcrfin': 102, 'gnttow': 103, 'gnwtea': 104,
    'gockin': 105, 'gocspa': 106, 'goleag': 107, 'grbher3': 108, 'grcfly': 109,
    'greegr': 110, 'greroa': 111, 'greyel': 112, 'grhowl': 113, 'grnher': 114,
    'grtgra': 115, 'grycat': 116, 'gryfly': 117, 'haiwoo': 118, 'hamfly': 119,
    'hergul': 120, 'herthr': 121, 'hoomer': 122, 'hoowar': 123, 'horgre': 124,
    'horlar': 125, 'houfin': 126, 'houspa': 127, 'houwre': 128, 'indbun': 129,
    'juntit1': 130, 'killde': 131, 'labwoo': 132, 'larspa': 133, 'lazbun': 134,
    'leabit': 135, 'leafly': 136, 'leasan': 137, 'lecthr': 138, 'lesgol': 139,
    'lesnig': 140, 'lesyel': 141, 'lewwoo': 142, 'linspa': 143, 'lobcur': 144,
    'lobdow': 145, 'logshr': 146, 'lotduc': 147, 'louwat': 148, 'macwar': 149,
    'magwar': 150, 'mallar3': 151, 'marwre': 152, 'merlin': 153, 'moublu': 154,
    'mouchi': 155, 'moudov': 156, 'norcar': 157, 'norfli': 158, 'norhar2': 159,
    'normoc': 160, 'norpar': 161, 'norpin': 162, 'norsho': 163, 'norwat': 164,
    'nrwswa': 165, 'nutwoo': 166, 'olsfly': 167, 'orcwar': 168, 'osprey': 169,
    'ovenbi1': 170, 'palwar': 171, 'pasfly': 172, 'pecsan': 173, 'perfal': 174,
    'phaino': 175, 'pibgre': 176, 'pilwoo': 177, 'pingro': 178, 'pinjay': 179,
    'pinsis': 180, 'pinwar': 181, 'plsvir': 182, 'prawar': 183, 'purfin': 184,
    'pygnut': 185, 'rebmer': 186, 'rebnut': 187, 'rebsap': 188, 'rebwoo': 189,
    'redcro': 190, 'redhea': 191, 'reevir1': 192, 'renpha': 193, 'reshaw': 194,
    'rethaw': 195, 'rewbla': 196, 'ribgul': 197, 'rinduc': 198, 'robgro': 199,
    'rocpig': 200, 'rocwre': 201, 'rthhum': 202, 'ruckin': 203, 'rudduc': 204,
    'rufgro': 205, 'rufhum': 206, 'rusbla': 207, 'sagspa1': 208, 'sagthr': 209,
    'savspa': 210, 'saypho': 211, 'scatan': 212, 'scoori': 213, 'semplo': 214,
    'semsan': 215, 'sheowl': 216, 'shshaw': 217, 'snobun': 218, 'snogoo': 219,
    'solsan': 220, 'sonspa': 221, 'sora': 222, 'sposan': 223, 'spotow': 224,
    'stejay': 225, 'swahaw': 226, 'swaspa': 227, 'swathr': 228, 'treswa': 229,
    'truswa': 230, 'tuftit': 231, 'tunswa': 232, 'veery': 233, 'vesspa': 234,
    'vigswa': 235, 'warvir': 236, 'wesblu': 237, 'wesgre': 238, 'weskin': 239,
    'wesmea': 240, 'wessan': 241, 'westan': 242, 'wewpew': 243, 'whbnut': 244,
    'whcspa': 245, 'whfibi': 246, 'whtspa': 247, 'whtswi': 248, 'wilfly': 249,
    'wilsni1': 250, 'wiltur': 251, 'winwre3': 252, 'wlswar': 253, 'wooduc': 254,
    'wooscj2': 255, 'woothr': 256, 'y00475': 257, 'yebfly': 258, 'yebsap': 259,
    'yehbla': 260, 'yelwar': 261, 'yerwar': 262, 'yetvir': 263
}

INV_BIRD_CODE = {v: k for k, v in BIRD_CODE.items()}

In [ ]:
tmp_list = []
for audio_d in TRAIN_RESAMPLED_AUDIO_DIRS:
    if not audio_d.exists():
        continue
    for ebird_d in audio_d.iterdir():
        if ebird_d.is_file():
            continue
        for wav_f in ebird_d.iterdir():
            tmp_list.append([ebird_d.name, wav_f.name, wav_f.as_posix()])
            
train_wav_path_exist = pd.DataFrame(
    tmp_list, columns=["ebird_code", "resampled_filename", "file_path"])

del tmp_list

train_all = pd.merge(
    train, train_wav_path_exist, on=["ebird_code", "resampled_filename"], how="inner")

print(train.shape)
print(train_wav_path_exist.shape)
print(train_all.shape)

del train
del train_wav_path_exist

skf = StratifiedKFold(**settings["split"]["params"])

train_all["fold"] = -1
for fold_id, (train_index, val_index) in enumerate(skf.split(train_all, train_all["ebird_code"])):
    train_all.iloc[val_index, -1] = fold_id
    

if DEVICE == 'tpu':
    from kaggle_datasets import KaggleDatasets
    GCS_PATH = KaggleDatasets().get_gcs_path('birdsong-resampled-train-audio-00')
    GCS_PATH1 = KaggleDatasets().get_gcs_path('birdsong-resampled-train-audio-01')
    GCS_PATH2 = KaggleDatasets().get_gcs_path('birdsong-resampled-train-audio-02')
    GCS_PATH3 = KaggleDatasets().get_gcs_path('birdsong-resampled-train-audio-03')
    GCS_PATH4 = KaggleDatasets().get_gcs_path('birdsong-resampled-train-audio-04')
    train_all['file_path'] = train_all['file_path'].str.replace('/kaggle/input/birdsong-resampled-train-audio-00', GCS_PATH)
    train_all['file_path'] = train_all['file_path'].str.replace('/kaggle/input/birdsong-resampled-train-audio-01', GCS_PATH1)
    train_all['file_path'] = train_all['file_path'].str.replace('/kaggle/input/birdsong-resampled-train-audio-02', GCS_PATH2)
    train_all['file_path'] = train_all['file_path'].str.replace('/kaggle/input/birdsong-resampled-train-audio-03', GCS_PATH3)
    train_all['file_path'] = train_all['file_path'].str.replace('/kaggle/input/birdsong-resampled-train-audio-04', GCS_PATH4)
    
use_fold = settings["globals"]["use_fold"]
train_file_list = train_all.query("fold != @use_fold")[["file_path", "ebird_code"]]
val_file_list = train_all.query("fold == @use_fold")[["file_path", "ebird_code"]]

print("[fold {}] train: {}, val: {}".format(use_fold, len(train_file_list), len(val_file_list)))

In [ ]:
train_file_list.ebird_code = train_file_list.ebird_code.map(BIRD_CODE)
val_file_list.ebird_code = val_file_list.ebird_code.map(BIRD_CODE)

In [ ]:
from random import shuffle
# def _get_examples_batch():
#   """Returns a shuffled batch of examples of all audio classes.
#   Note that this is just a toy function because this is a simple demo intended
#   to illustrate how the training code might work.
#   Returns:
#     a tuple (features, labels) where features is a NumPy array of shape
#     [batch_size, num_frames, num_bands] where the batch_size is variable and
#     each row is a log mel spectrogram patch of shape [num_frames, num_bands]
#     suitable for feeding VGGish, while labels is a NumPy array of shape
#     [batch_size, num_classes] where each row is a multi-hot label vector that
#     provides the labels for corresponding rows in features.
#   """
#   # Make a waveform for each class.
#   num_seconds = 5
#   sr = 44100  # Sampling rate.
#   t = np.arange(0, num_seconds, 1 / sr)  # Time axis
#   # Random sine wave.
#   freq = np.random.uniform(100, 1000)
#   sine = np.sin(2 * np.pi * freq * t)
#   # Random constant signal.
#   magnitude = np.random.uniform(-1, 1)
#   const = magnitude * t
#   # White noise.
#   noise = np.random.normal(-1, 1, size=t.shape)

#   # Make examples of each signal and corresponding labels.
#   # Sine is class index 0, Const class index 1, Noise class index 2.
#   sine_examples = waveform_to_examples(sine, sr)
#   sine_labels = np.array([[1, 0, 0]] * sine_examples.shape[0])
#   const_examples = waveform_to_examples(const, sr)
#   const_labels = np.array([[0, 1, 0]] * const_examples.shape[0])
#   noise_examples = waveform_to_examples(noise, sr)
#   noise_labels = np.array([[0, 0, 1]] * noise_examples.shape[0])

#   # Shuffle (example, label) pairs across all classes.
#   all_examples = np.concatenate((sine_examples, const_examples, noise_examples))
#   all_labels = np.concatenate((sine_labels, const_labels, noise_labels))
#   labeled_examples = list(zip(all_examples, all_labels))
#   shuffle(labeled_examples)

#   # Separate and return the features and labels.
#   features = [example for (example, _) in labeled_examples]
#   labels = [label for (_, label) in labeled_examples]
#   return (features, labels)

# all_examples , labeled_examples = _get_examples_batch()

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, 
                 dim=[(settings["globals"]['target_sr'] * 5)//2], n_channels=1,
                 n_classes=264, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        
    
    
    
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
#         list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = []
        y = []

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
#             print(ID)
            _tempx = wavfile_to_examples(self.list_IDs[ID])            
            X.extend(_tempx)

            # Store class
            labels = np.zeros(len(BIRD_CODE), dtype="f")
            labels[self.labels[ID]] = 1
            y.extend([labels] * _tempx.shape[0])
        labeled_examples = list(zip(X, y))
        shuffle(labeled_examples)
        # Separate and return the features and labels.
        features = [example for (example, _) in labeled_examples]
        labels = [label for (_, label) in labeled_examples]
        return features, labels

In [ ]:
X, Y = train_file_list['file_path'].values.tolist(), train_file_list['ebird_code'].values.tolist()

In [ ]:
training_generator = DataGenerator(X, Y,batch_size=32)

In [ ]:
# x, y = training_generator[1]

In [ ]:
# !curl -O https://storage.googleapis.com/audioset/vggish_model.ckpt
# !curl -O https://storage.googleapis.com/audioset/vggish_pca_params.npz

In [ ]:
# def del_all_flags(FLAGS):
#     flags_dict = FLAGS._flags()
#     keys_list = [keys for keys in flags_dict]
#     for keys in keys_list:
#     FLAGS.delattr(keys)

# del_all_flags(tf.flags.FLAGS)


# flags = tf.app.flags

# flags.DEFINE_integer(
#     'num_batches', 30,
#     'Number of batches of examples to feed into the model. Each batch is of '
#     'variable size and contains shuffled examples of each class of audio.')

# flags.DEFINE_boolean(
#     'train_vggish', True,
#     'If True, allow VGGish parameters to change during training, thus '
#     'fine-tuning VGGish. If False, VGGish parameters are fixed, thus using '
#     'VGGish as a fixed feature extractor.')

# flags.DEFINE_string(
#     'checkpoint', 'vggish_model.ckpt',
#     'Path to the VGGish checkpoint file.')

# FLAGS = flags.FLAGS

_NUM_CLASSES = 264

In [ ]:
import tqdm
_NUM_CLASSES = 264
with tf.Graph().as_default(), tf.Session() as sess:
    # Define VGGish.
#     embeddings = define_vggish_slim(training=FLAGS.train_vggish)
    embeddings = define_vggish_slim(training=True)

    # Define a shallow classification model and associated training ops on top
    # of VGGish.
    with tf.variable_scope('mymodel'):
      # Add a fully connected layer with 100 units. Add an activation function
      # to the embeddings since they are pre-activation.
      num_units = 100
      fc = slim.fully_connected(tf.nn.relu(embeddings), num_units)

      # Add a classifier layer at the end, consisting of parallel logistic
      # classifiers, one per class. This allows for multi-class tasks.
      logits = slim.fully_connected(
          fc, _NUM_CLASSES, activation_fn=None, scope='logits')
      tf.sigmoid(logits, name='prediction')
      
      # Add training ops.
      with tf.variable_scope('train'):
        global_step = tf.train.create_global_step()

        # Labels are assumed to be fed as a batch multi-hot vectors, with
        # a 1 in the position of each positive class label, and 0 elsewhere.
        labels_input = tf.placeholder(
            tf.float32, shape=(None, _NUM_CLASSES), name='labels')

        # Cross-entropy label loss.
        xent = tf.nn.sigmoid_cross_entropy_with_logits(
            logits=logits, labels=labels_input, name='xent')
        loss = tf.reduce_mean(xent, name='loss_op')
        tf.summary.scalar('loss', loss)

        # We use the same optimizer and hyperparameters as used to train VGGish.
        optimizer = tf.train.AdamOptimizer(
            learning_rate=LEARNING_RATE,
            epsilon=ADAM_EPSILON)
        train_op = optimizer.minimize(loss, global_step=global_step)

    # Initialize all variables in the model, and then load the pre-trained
    # VGGish checkpoint.
    sess.run(tf.global_variables_initializer())
#     vggish_slim.load_vggish_slim_checkpoint(sess, FLAGS.checkpoint)
#     load_vggish_slim_checkpoint(sess, 'vggish_model.ckpt')
    load_vggish_slim_checkpoint(sess, '../input/tensorflowbirdcallvggish2epoch/saved_vggish1/vggish')
    

    # The training loop.
    features_input = sess.graph.get_tensor_by_name(
        INPUT_TENSOR_NAME)
    for epoch_id in range(settings['globals']['num_epochs']):
        print(f"epoch {epoch_id} started")
    #     for _ in range(FLAGS.num_batches):
        for batch_idx in range(len(training_generator)):
          (features, labels) = training_generator[batch_idx]
          [num_steps, loss_value, _] = sess.run(
              [global_step, loss, train_op],
              feed_dict={features_input: features, labels_input: labels})
          print('Step %d: loss %g' % (num_steps, loss_value))
        saver = tf.train.Saver()
        directory = f'./saved_vggish{epoch_id}/'
        filename = 'vggish'
        if not os.path.exists(directory):
            os.makedirs(directory)
        filepath = os.path.join(directory, filename)
        saver.save(sess, filepath)

In [ ]:
import gc
gc.collect()

In [ ]:
# tf.train.latest_checkpoint('../input/tensorflowbirdcallvggish2epoch/saved_vggish1/')

It will stuck after some data see here
https://github.com/keras-team/keras/issues/10948

This will cause deadlock problem

use_multiprocessing=True,

workers=2,

In [ ]:
# model = compile_new_model()
# history = model.fit_generator(generator=training_generator,
#                     epochs=settings['globals']['num_epochs'],
# #                     validation_data=validation_generator,
# #                     use_multiprocessing=True,
# #                     workers=2,
#                     callbacks=[earlystopping_cb, mdlcheckpoint_cb])

In [ ]:
# model = compile_new_model()
# history = model.fit(
#     train_ds,
#     epochs=settings['globals']['num_epochs'],
#     validation_data=valid_ds,
#     callbacks=[earlystopping_cb, mdlcheckpoint_cb],
# )